In [53]:
!sudo apt update
!sudo apt install -y libcairo2-dev libpango1.0-dev ffmpeg
# Install a stable version of Manim that works in Colab
!pip install manim==0.17.3
!pip install pydantic==1.10.7
!pip install typing-extensions==4.5.0
!pip install typing

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 261 kB in 2s (163 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
44 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' a

  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.13.0
    Uninstalling typing_extensions-4.13.0:
      Successfully uninstalled typing_extensions-4.13.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
typeguard 4.4.2 requires typing_extensions>=4.10.0, but you have typing-extensions 4.5.0 which is incompatible.
sqlalchemy 2.0.40 requires typing-extensions>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
albumentations 2.0.5 requires pydantic>=2.9.2, but you have pydantic 1.10.7 which is incompatible.
langchain 0.3.22 requires pydantic<3.0.0,>=2.7.4, but you have pydantic 1.10.7 which is incompatible.
altair 5.5.0 requires typing-extensions>=4.10.0; python_version < "3.14", but you have typing-extensions 4.5.0 which is incompatible.
langchain-core 0.3.49 requires pydantic<3.0.0,>=2.5.2; python_full

In [1]:
# Load the Manim extension to use %%manim magic
%load_ext manim

Manim Community v0.17.3

The manim module is not an IPython extension.


In [21]:
from pydantic import BaseModel
from typing import List, Optional

class Unit(BaseModel):
    id: str
    name: str
    unit_type: str  # e.g. "infantry", "armor", "air support"
    strength: int  # combat effectiveness, 0–100
    position: Optional[tuple[float, float]] = None  # (x, y) coords
    allegiance: str  # "friendly" or "enemy"
    status: str = "active"  # e.g. "active", "retreating", "destroyed"

class TerrainFeature(BaseModel):
    type: str  # e.g. "hill", "forest", "building"
    position: tuple[float, float]
    size: float  # area/radius in meters

class Terrain(BaseModel):
    type: str  # e.g. "urban", "desert", "jungle"
    features: List[TerrainFeature]
    dimensions: tuple[int, int]  # map size in meters (width, height)

class Objective(BaseModel):
    id: str
    description: str
    controlling_unit_ids: List[str] = []
    completed: bool = False
    location: Optional[tuple[float, float]]
    priority: int = 1  # Higher number = more critical

class BattleEvent(BaseModel):
    timestamp: object  # e.g. "00:05", "12:03 PM"
    description: str
    involved_units: List[str] = []
    event_type: str  # e.g. "move", "fire", "retreat", "reinforce"

class Scenario(BaseModel):
    title: str
    description: str
    terrain: Terrain
    units: List[Unit]
    objectives: List[Objective]
    timeline: List[BattleEvent]

In [22]:
#from models import *

# ------------------------
# Parser Function
# ------------------------
test_input = """
Title:Operation Dynamo
Description:Allied forces are retreating and preparing for naval evacuation under fire from advancing Axis troops.
Unit:ID=U1, Name=British Infantry, Type=infantry, Strength=85, Allegiance=friendly, X=-3, Y=-2.5
Unit:ID=U2, Name=French Infantry, Type=infantry, Strength=78, Allegiance=friendly, X=-1, Y=-2.2
Unit:ID=U3, Name=German Armor, Type=armor, Strength=92, Allegiance=enemy, X=2, Y=-1.8
Feature:Type=Bunker, X=0, Y=-2, Size=10
Objective:ID=O1, Desc=Evacuate to naval boats, X=4, Y=0.5, Priority=1
Event:Time=00:00, Desc=British Infantry begins fallback to coast, Units=U1, Type=move
Event:Time=00:01, Desc=German Armor advances toward beach, Units=U3, Type=move
"""


def parse_scenario_text(text: str) -> Scenario:
    lines = [line.strip() for line in text.strip().split("\n") if line.strip()]
    units = []
    features = []
    objectives = []
    timeline = []
    title = "Generated Scenario"
    description = ""
    terrain_type = "beach"
    terrain_dims = (1000, 1000)

    for line in lines:
        if line.startswith("Title:"):
            title = line.split(":", 1)[1].strip()
        elif line.startswith("Description:"):
            description = line.split(":", 1)[1].strip()
        elif line.startswith("Unit:"):
            props = dict(item.split("=") for item in line.split(":", 1)[1].split(", "))
            unit = Unit(
                id=props["ID"],
                name=props["Name"],
                unit_type=props["Type"],
                strength=int(props["Strength"]),
                allegiance=props["Allegiance"],
                position=(float(props.get("X", 0)), float(props.get("Y", 0)))
            )
            units.append(unit)
        elif line.startswith("Feature:"):
            props = dict(item.split("=") for item in line.split(":", 1)[1].split(", "))
            features.append(TerrainFeature(
                type=props["Type"],
                position=(float(props["X"]), float(props["Y"])),
                size=float(props["Size"])
            ))
        elif line.startswith("Objective:"):
            props = dict(item.split("=") for item in line.split(":", 1)[1].split(", "))
            objectives.append(Objective(
                id=props["ID"],
                description=props["Desc"],
                location=(float(props.get("X", 0)), float(props.get("Y", 0))),
                priority=int(props.get("Priority", 1))
            ))
        elif line.startswith("Event:"):
            props = dict(item.split("=") for item in line.split(":", 1)[1].split(", "))
            timeline.append(BattleEvent(
                timestamp=props["Time"],
                description=props["Desc"],
                involved_units=props["Units"].split("|"),
                event_type=props["Type"]
            ))

    return Scenario(
        title=title,
        description=description,
        terrain=Terrain(type=terrain_type, features=features, dimensions=terrain_dims),
        units=units,
        objectives=objectives,
        timeline=timeline
    )



In [4]:
%%manim -qm DunkirkBeachScene

from manim import *

class DunkirkBeachScene(Scene):
    def construct(self):

        # Set white background
        background = Rectangle(width=20, height=12, fill_color=BLACK, fill_opacity=1).set_z_index(-1)
        self.add(background)

        # Load SVG representing France map
        france_svg = SVGMobject("sample_data/nord-france.svg")
        france_svg.scale(2.5)
        france_svg.set_fill(YELLOW_E, opacity=0.3)
        france_svg.set_stroke(YELLOW_E)
        self.add(france_svg)

        # Title
        title = Text("Dunkirk Evacuation - Visualization").scale(0.6).to_edge(UP)
        self.add(title)

        # Units - manually placed on Dunkirk coastal region approximation
        british_troop = Dot(point=[1, 2, 0], radius=0.15, color=GREEN)
        british_label = Text("British Infantry").scale(0.4).next_to(british_troop, UP)

        french_troop = Dot(point=[1, 1.5, 0], radius=0.15, color=BLUE)
        french_label = Text("French Infantry").scale(0.4).next_to(french_troop, UP)

        german_troop = Dot(point=[0.5, 1, 0], radius=0.15, color=RED)
        german_label = Text("German Forces").scale(0.4).next_to(german_troop, UP)

        self.add(british_troop, british_label, french_troop, french_label, german_troop, german_label)

        # Evacuation Zone
        evac_zone = Square(side_length=1.5, color=WHITE, fill_opacity=0.2).move_to([2, 2, 0])
        evac_label = Text("Evacuation Zone").scale(0.3).next_to(evac_zone, DOWN)
        self.add(evac_zone, evac_label)

        # Animate British movement to evacuation zone
        self.wait(1)
        self.play(british_troop.animate.move_to([2, 2, 0]), run_time=3)
        self.wait(1)

        # Final message
        complete_label = Text("Evacuation Successful").scale(0.5).next_to(evac_zone, UP).set_color(GREEN)
        self.play(FadeIn(complete_label))
        self.wait(2)


[04/06/25 05:19:21] INFO     Animation 0 : Partial movie file written in                   ]8;id=761523;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=782726;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/content/media/videos/content/720p30/partial_movie_files/Dun                         
                             kirkBeachScene/274514146_1076466279_620829335.mp4'                                    

INFO:manim:Animation 0 : Partial movie file written in '/content/media/videos/content/720p30/partial_movie_files/DunkirkBeachScene/274514146_1076466279_620829335.mp4'


[04/06/25 05:19:25] INFO     Animation 1 : Partial movie file written in                   ]8;id=579723;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=109709;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/content/media/videos/content/720p30/partial_movie_files/Dun                         
                             kirkBeachScene/2789044632_2669002150_1871908949.mp4'                                  

INFO:manim:Animation 1 : Partial movie file written in '/content/media/videos/content/720p30/partial_movie_files/DunkirkBeachScene/2789044632_2669002150_1871908949.mp4'


[04/06/25 05:19:27] INFO     Animation 2 : Partial movie file written in                   ]8;id=238443;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=766828;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/content/media/videos/content/720p30/partial_movie_files/Dun                         
                             kirkBeachScene/2789044632_345352296_1634755845.mp4'                                   

...

INFO:manim:Combining to Movie file.


                    INFO                                                                   ]8;id=533145;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=749851;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py#736\736]8;;\
                             File ready at                                                                         
                             '/content/media/videos/content/720p30/DunkirkBeachScene.mp4'                          
                                                                                                                   

INFO:manim:
File ready at '/content/media/videos/content/720p30/DunkirkBeachScene.mp4'



                    INFO     Rendered DunkirkBeachScene                                                ]8;id=397848;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=787162;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene.py#241\241]8;;\
                             Played 5 animations                                                                   

INFO:manim:Rendered DunkirkBeachScene
Played 5 animations


In [ ]:
### Below is an example with shapes verse a map


In [ ]:
%%manim -qm DunkirkBeachScene

from manim import *
class DunkirkBeachScene(Scene):
    def construct(self):
        # Draw stylized beach and water using shapes
        beach = Rectangle(width=12, height=4, fill_color=YELLOW_E, fill_opacity=0.4).move_to([0, -2, 0])
        sea = Rectangle(width=12, height=3, fill_color=BLUE, fill_opacity=0.3).next_to(beach, UP, buff=0)
        self.add(sea, beach)

        # Title
        title = Text("Dunkirk Evacuation - Visualization").scale(0.6).to_edge(UP)
        self.add(title)

        # Units - positioned on beach
        british_troop = Dot(point=[-3, -2.5, 0], radius=0.15, color=GREEN)
        british_label = Text("British Infantry").scale(0.4).next_to(british_troop, UP)

        french_troop = Dot(point=[-1, -2.2, 0], radius=0.15, color=BLUE)
        french_label = Text("French Infantry").scale(0.4).next_to(french_troop, UP)

        german_troop = Dot(point=[2, -1.8, 0], radius=0.15, color=RED)
        german_label = Text("German Forces").scale(0.4).next_to(german_troop, UP)

        self.add(british_troop, british_label, french_troop, french_label, german_troop, german_label)

        # Evacuation Zone (placed in the sea)
        evac_zone = Square(side_length=1.5, color=WHITE, fill_opacity=0.2).move_to([4, 0.5, 0])
        evac_label = Text("Evacuation Zone").scale(0.3).next_to(evac_zone, DOWN)
        self.add(evac_zone, evac_label)

        # Animate British movement to evacuation zone
        self.wait(1)
        self.play(british_troop.animate.move_to([4, 0.5, 0]), run_time=3)
        self.wait(1)

        # Final message
        complete_label = Text("Evacuation Successful").scale(0.5).next_to(evac_zone, UP).set_color(GREEN)
        self.play(FadeIn(complete_label))
        self.wait(2)


[04/05/25 16:40:31] INFO     Animation 0 : Partial movie file written in                   ]8;id=252612;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=323568;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/content/media/videos/content/720p30/partial_movie_files/Dun                         
                             kirkBeachScene/274514146_1076466279_140515962.mp4'                                    

INFO:manim:Animation 0 : Partial movie file written in '/content/media/videos/content/720p30/partial_movie_files/DunkirkBeachScene/274514146_1076466279_140515962.mp4'


[04/05/25 16:40:36] INFO     Animation 1 : Partial movie file written in                   ]8;id=239740;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=505066;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/content/media/videos/content/720p30/partial_movie_files/Dun                         
                             kirkBeachScene/2789044632_445026133_1359018374.mp4'                                   

INFO:manim:Animation 1 : Partial movie file written in '/content/media/videos/content/720p30/partial_movie_files/DunkirkBeachScene/2789044632_445026133_1359018374.mp4'


[04/05/25 16:40:37] INFO     Animation 2 : Partial movie file written in                   ]8;id=740743;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=423864;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/content/media/videos/content/720p30/partial_movie_files/Dun                         
                             kirkBeachScene/2789044632_345352296_1223585236.mp4'                                   

...

INFO:manim:Combining to Movie file.


                    INFO                                                                   ]8;id=533519;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=465952;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py#736\736]8;;\
                             File ready at                                                                         
                             '/content/media/videos/content/720p30/DunkirkBeachScene.mp4'                          
                                                                                                                   

INFO:manim:
File ready at '/content/media/videos/content/720p30/DunkirkBeachScene.mp4'



                    INFO     Rendered DunkirkBeachScene                                                ]8;id=611154;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=541420;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene.py#241\241]8;;\
                             Played 5 animations                                                                   

INFO:manim:Rendered DunkirkBeachScene
Played 5 animations


**The Manim scene now supports "fire" and "hold" events**

In [25]:
%%manim -qm DunkirkBeachScene
from manim import *

class DunkirkBeachScene(Scene):
    def construct(self):
        # Input text
        text_input = """
        Title:Operation Dynamo
        Description:Allied forces are retreating and preparing for naval evacuation under fire from advancing Axis troops.
        Unit:ID=U1, Name=British Infantry, Type=infantry, Strength=85, Allegiance=friendly, X=-3, Y=-2.5
        Unit:ID=U2, Name=French Infantry, Type=infantry, Strength=78, Allegiance=friendly, X=-1, Y=-2.2
        Unit:ID=U3, Name=German Armor, Type=armor, Strength=92, Allegiance=enemy, X=2, Y=-1.8
        Feature:Type=Bunker, X=0, Y=-2, Size=10
        Objective:ID=O1, Desc=Evacuate to naval boats, X=4, Y=0.5, Priority=1
        Event:Time=00:00, Desc=British Infantry begins fallback to coast, Units=U1, Type=move
        Event:Time=00:01, Desc=German Armor fires on beach, Units=U3, Type=fire
        Event:Time=00:02, Desc=French Infantry holds position, Units=U2, Type=hold
        """

        scenario = parse_scenario_text(text_input)

        beach = Rectangle(width=12, height=4, fill_color=YELLOW_E, fill_opacity=0.4).move_to([0, -2, 0])
        sea = Rectangle(width=12, height=3, fill_color=BLUE, fill_opacity=0.3).next_to(beach, UP, buff=0)
        self.add(sea, beach)

        title = Text(scenario.title).scale(0.6).to_edge(UP)
        self.add(title)

        unit_dots = {}
        for unit in scenario.units:
            if unit.name == "British Infantry":
                color = BLUE
            elif unit.name == "French Infantry":
                color = GREEN
            elif unit.allegiance == "enemy":
                color = RED
            if unit.unit_type == "infantry":
                dot = Dot(point=[unit.position[0], unit.position[1], 0], radius=0.15, color=color)
            elif unit.unit_type == "armor":
                dot = Square(side_length=0.3, color=color).move_to([unit.position[0], unit.position[1], 0])
            else:
                dot = Dot(point=[unit.position[0], unit.position[1], 0], radius=0.15, color=color)
            label = Text(unit.name).scale(0.4).next_to(dot, UP)
            self.add(dot, label)
            unit_dots[unit.id] = dot

        if scenario.objectives:
            evac = scenario.objectives[0]
            evac_zone = Square(side_length=1.5, color=WHITE, fill_opacity=0.2).move_to([evac.location[0], evac.location[1], 0])
            evac_label = Text("Evacuation Zone").scale(0.3).next_to(evac_zone, DOWN)
            self.add(evac_zone, evac_label)

        self.wait(1)
        for event in scenario.timeline:
            for uid in event.involved_units:
                if uid in unit_dots:
                    dot = unit_dots[uid]
                    if event.event_type == "move":
                        self.play(dot.animate.move_to([evac.location[0], evac.location[1], 0]), run_time=2)
                    elif event.event_type == "fire":
                        self.play(Flash(dot), run_time=1)
                    elif event.event_type == "hold":
                        self.play(dot.animate.scale(1.2).scale(1 / 1.2), run_time=1)

        complete_label = Text("Evacuation Complete").scale(0.5).next_to(evac_zone, UP).set_color(GREEN)
        self.play(FadeIn(complete_label))
        self.wait(2)


[04/06/25 07:14:25] INFO     Animation 0 : Partial movie file written in                   ]8;id=710046;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=176724;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/content/media/videos/content/720p30/partial_movie_files/Dun                         
                             kirkBeachScene/274514146_1076466279_1673751381.mp4'                                   

INFO:manim:Animation 0 : Partial movie file written in '/content/media/videos/content/720p30/partial_movie_files/DunkirkBeachScene/274514146_1076466279_1673751381.mp4'


[04/06/25 07:14:28] INFO     Animation 1 : Partial movie file written in                   ]8;id=281437;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=747601;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/content/media/videos/content/720p30/partial_movie_files/Dun                         
                             kirkBeachScene/2789044632_665480590_4040049057.mp4'                                   

INFO:manim:Animation 1 : Partial movie file written in '/content/media/videos/content/720p30/partial_movie_files/DunkirkBeachScene/2789044632_665480590_4040049057.mp4'


[04/06/25 07:14:30] INFO     Animation 2 : Partial movie file written in                   ]8;id=685613;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=986132;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/content/media/videos/content/720p30/partial_movie_files/Dun                         
                             kirkBeachScene/2789044632_3841203318_1660894343.mp4'                                  

...

INFO:manim:Combining to Movie file.


                    INFO                                                                   ]8;id=215872;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=511140;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene_file_writer.py#736\736]8;;\
                             File ready at                                                                         
                             '/content/media/videos/content/720p30/DunkirkBeachScene.mp4'                          
                                                                                                                   

INFO:manim:
File ready at '/content/media/videos/content/720p30/DunkirkBeachScene.mp4'



                    INFO     Rendered DunkirkBeachScene                                                ]8;id=2752;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=675597;file:///usr/local/lib/python3.11/dist-packages/manim/scene/scene.py#241\241]8;;\
                             Played 6 animations                                                                   

INFO:manim:Rendered DunkirkBeachScene
Played 6 animations
